# MS 141 Lecture 13 

# Exploring Partial Differential Equations

## Read: Chapter 9 of Newman's book

This lecture uses codes from a NumFys notebook by Hugdal et al. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as linalg

%matplotlib inline

# Set common figure parameters
newparams = {'figure.figsize': (10, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 24}
plt.rcParams.update(newparams)

To conclude our introduction to scientific computing, we explore partial differential equations. 

A *partial differential equation (PDE)* is a relation among the **partial derivatives of an unknown function of multiple variables, the unknown function, and the independent variables**. Just as in the case of ODEs, the order of the PDE is defined by the highest derivative, and we say that a PDE is linear if the dependent variable (the unknown function) and its partial derivatives all appear with a power of one. 

**PDEs are usually classified into families**. For each family, besides analytic solution approaches there are numerical methods that can solve the PDE with a given accuracy. Many PDEs cannot be solved or are challenging to solve analytically. Therefore, numerical methods are widely used to solve PDEs and constitute an important component of scientific computing in science and engineering.

The three families of PDEs include (we call $u$ the unknown function):
-  **Parabolic**, which describe diffusion and transport by conduction. A typical PDE is:

$$~~~~~~~~~~~\frac{\partial u}{\partial t} = D \frac{\partial^2 u}{\partial x^2}~~~~ (\text{Diffusion or heat equation})$$

-  **Hyperbolic**, which describe the motion of a wave or fluid. Examples are:

$$~~\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0~~~~ (\text{Advection equation)}$$

$$\frac{\partial^2 u}{\partial t^2} - v^2 \frac{\partial^2 u}{\partial x^2} = 0~~~~ (\text{Wave equation)}$$

-  **Elliptic**, typically a steady-state boundary value problem determined by a conservation law. Time is usually not involved. Examples are:

$$~~\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = 0~~~~ (\text{Laplace's equation)}$$

In each case, one needs to define appropriate initial and/or boundary conditions to make the problem well-defined. The most common are Dirichlet<br> ($u$ fixed at a given boundary), Neumann (derivative of $u$ fixed at a given boundary) and Robin (a combination of $u$ and its derivative is fixed at a domain boundary).

There are **three main numerical solution methods for PDEs: finite difference, finite elements, and finite volume**.<br> We will **focus on finite-difference methods**, which extends to multiple variables the approach we saw in the ODE lectures. The finite-difference approach discretizes the time and space domains and solves the PDE at each grid point using finite-difference approximations of the derivatives.

When solving a PDE numerically, deriving the discretized version of the equation is the main challenge. There are many solution methods and discretization schemes, each with its own strenghts and weaknesses in terms of accuracy, stability and computational cost. Specific numerical approaches are employed for each family of PDEs (parabolic, elliptic, and hyperbolic).

We will discuss three examples of discretizing and solving numerically the following PDEs: 

- Elliptic: The 2-dimensional Laplace equation
    $$ \left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}\right)u = 0$$
    
- Parabolic: The 1-dimensional heat equation
    $$ \frac{\partial u}{\partial t} = D \frac{\partial^2u}{\partial x^2}$$
    
- Hyperbolic: The advection equation (next lecture)
    $$ \frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0\,\,$$

## Laplace Equation in 2D
The Laplace equation:

$$ \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = 0$$

can be discretized using the central difference scheme, which is second order accurate. We first discretize the $x$- and $y$-axes using $x_i = x_{min} + i\Delta x$, $i\in[0,N_x]$, and $y_j = y_{min} + j\Delta y$, $j\in[0,N_y]$, where $\Delta x = (x_{max}-x_{min})/N_x$ and $\Delta y = (y_{max}-y_{min})/N_y$. This scheme thus discretizes $u$ as $u(x_i, y_j) = u_{i,j}$. 

The second derivative with respect to $x$ becomes

$$\frac{\partial^2 u}{\partial x^2} \rightarrow \frac{u_{i+1,j} - 2u_{i,j} + u_{i-1,j}}{\Delta x^2},$$

and similarly for the second derivative with respect to $y$,

$$\frac{\partial^2 u}{\partial y^2} \rightarrow \frac{u_{i,j+1} - 2u_{i,j} + u_{i,j-1}}{\Delta y^2}.$$ 

The discretized Laplace equation is then written as
$$\frac{u_{i+1,j} - 2u_{i,j} + u_{i-1,j}}{\Delta x^2} +
\frac{u_{i,j+1} - 2u_{i,j} + u_{i,j-1}}{\Delta y^2} = 0.$$

We solve the above equation on the domain $x\in[0,1]$, $y\in[0,1]$, with the following boundary conditions (BCs):

- $u(x,0) = u_a = 5$

- $u(x,1) = u_b(x) = 5(1-\sin(\pi x))$

- $u(0,y) = u_c = 5$

- $\frac{\partial u(x,y)}{\partial x}\Big|_{x=1} = 0$.

The first three of these boundary conditions are Dirichlet BCs, while the last one is a Neumann BC, which requires special care since it doesn't give information about the value at the boundary directly.
<img src="https://www.numfys.net/media/notebooks/images/mo_pdes1_laplace_grid.png" alt="Grid" style="width:604px">

The above image shows how we can choose to discretize the domain. Similar to what we saw in the boundary value ODE problem, we don't include the boundary points at which we know the value of $u$, i.e. the points on the $x$- and $y$-axis, and the points on the line $y=1$. However, we include the points on the line $x=1$, since here the value of $u$ is unknown. This means that we have the following discretization of the $x$- and $y$-axes:

- $x_i = (1+i)\Delta x$, with $\Delta x = \frac{1}{N_x+1}$ and $i\in [0, N_x]$.

- $y_j = (1+j)\Delta y$, with $\Delta y = \frac{1}{N_y+2}$ and $j \in [0, N_y]$.

Let us now treat the various boundary cases:

- For $i=0$ we get
    $$\frac{u_{0,j+1} -2u_{0,j} + u_{0,j-1}}{\Delta y^2} + \frac{u_{1,j} -2u_{0,j} + u_c}{\Delta x^2} = 0,$$
    where $u_{-1,j}$ has been replaced by the value at the left boundary, $u_c$.
    
- For $j=0$ we get
    $$\frac{u_{i,0} -2u_{i,0} + u_a}{\Delta y^2} + \frac{u_{i+1,0} -2u_{i,0} + u_{i-1,0}}{\Delta x^2} = 0,$$
    where $u_{i,-1}$ has been replaced by the value at the bottom boundary, $u_a$.
- For $j=N_y$ we get
    $$\frac{u_b(x_i) -2u_{i,N_y} + u_{i,N_y-1}}{\Delta y^2} + \frac{u_{i+1,N_y} -2u_{i,N_y} + u_{i-1,N_y}}{\Delta x^2} = 0,$$
    where $u_{i,N_y+1}$ has been replaced by the function giving the values at the top boundary, $u_b(x_i)$.
-  For $i=N_x$ we get
      $$\frac{u_{N_x,j+1} -2u_{N_x,j} + u_{N_x,j-1}}{\Delta y^2} + \frac{u_{N_x+1,j} -2u_{N_x,j} + u_{N_x-1,j}}{\Delta x^2} = 0.$$
      
      Here we run into a problem: we don't have a value for $u_{N_x+1,j}$, which is a point outside the domain. However, we can find one using the boundary condition: we discretize the boundary condition,
    $$ \frac{\partial u(x,y)}{\partial x} \rightarrow \frac{u_{i+1,j} - u_{i-1,j}}{2\Delta x},$$
    which gives at the boundary
    $$\frac{u_{N+1,j} - u_{N-1,j}}{2\Delta x} = 0.$$
    
    Hence we get that $u_{N+1,j} = u_{N-1,j}$, and get the following equation at the right boundary:
    
    $$\frac{u_{N_x,j+1} -2u_{N_x,j} + u_{N_x,j-1}}{\Delta y^2} + \frac{-2u_{N_x,j} + 2u_{N_x-1,j}}{\Delta x^2} = 0.$$
    Notice that we get a factor $2$ in front of $u_{N_x-1,j}$.

In the above, all the constants will be moved to the right hand side, forming the vector ${\bf{b}}$ in the matrix equation

$$A{\bf{u}} = {\bf{b}}.$$

Note also that in the corners of the domain, i.e. at the points $(i,j) = (0,0)$, $(N_y,0)$, $(0,N_x)$ and $(N_y,N_x)$, we need to combine two of the above boundary conditions.

We can now start constructing the matrix equation using these results. For simplicity, we set $\Delta x = \Delta y$, which simplifies the stencil. We get
$$\left(\matrix{
-4 & 1 & 0 &...& 0 & 1 & 0 &0&...\\
1 & -4 & 1 &...& 0 & 0 & 1 &0&...\\
0& 1 & -4 & ...& 0 &0&0&1&...\\
\vdots& \vdots& \vdots& \vdots& \vdots& \vdots &\vdots&\vdots&\vdots&\ddots&
}\right)
\left(\matrix{
u_{0,0}\\
u_{1,0}\\
u_{2,0}\\
\vdots\\
u_{N_x,0}\\
u_{0,1}\\
u_{1,1}\\
u_{2,1}\\
\vdots
}\right) = \left(\matrix{
-u_c-u_a\\
-u_a\\
-u_a\\ \vdots}\right)$$
Here A is a pentadiagonal $N\times N$-matrix with size $N=N_x\cdot N_y$, and ${\bf b}$ is a vector with length $N$.

The related code is shown below. We constuct the matrix $A$ and vector ${\bf b}$. It can be difficult to get all the elements right; it's a good idea to discretize the system using just a few grid points, and then check the implementation using this small grid.

In [ ]:
# Set the number of grid points
Nx = 50
dx = 1/(Nx+1)
x = np.linspace(dx, 1, Nx+1)

dy = dx
Ny = int(1/dy - 2) #Nx - 1
N = (Nx+1)*(Ny+1)
y = np.linspace(dy, 1-dy, Ny+1)

# Set constants at boundary
ua = 5
ub = 5*(1-np.sin(np.pi*x))
uc = 5

### Construct the matrix A
A = np.zeros([N,N])

for i in range(N):
    # Set the diagonals to -4
    A[i, i] = -4
    # Set the non-zero off-diagonals to 1
    if i > 0:
        A[i, i-1] = 1
    if i < N-1:
        A[i, i+1] = 1
    if i < N-Nx-1: # similar to above, but translated by Nx
        A[i+Nx+1, i] = 1
        A[i, i+Nx+1] = 1
    
for i in range(N):
    # Some elements must be changed to 2 due to the Neumann BCs
    if (i+1)%(Nx+1) == 0:
        A[i,i-1] = 2
    
    # In addition some elements must be set to zero, since
    # the elements are taken care of by terms in b
    if i > 0 and i < N - 1 and (i+1)%(Nx+1) == 0:
        A[i, i+1] = 0
        A[i+1, i] = 0

### Construct the right hand side vector b:
b = np.zeros(N)

# For j = 0, set elements to -ua
b[0:Nx+1] = - ua

# For i = 0, set the elements to -uc
indices = np.mod(range(N),Nx+1) == 0
b[indices] += -uc

# For j = Ny, set elements to -ub(x_i)
for i in range(Nx+1):
    b[N-Nx-1+i] += -ub[i]

Finally, we solve the matrix equation using ``linalg.solve()``, and plot the result.

In [ ]:
# Solve the system using linalg
u = linalg.solve(A,b)

# Reshape solution array to be able to plot it
u_ = u.reshape(Ny+1,Nx+1)
plt.figure(figsize=(10, 8))
plt.contourf(x, y, u_, 100)
plt.colorbar()
plt.ylabel('y')
plt.xlabel('x')
plt.show()

## Heat equation in 1D
As a first step, we discretize the 1D heat equation:

$$ \frac{\partial u}{\partial t} = \alpha \frac{\partial^2u}{\partial x^2}$$

in which the unknown function $u$ represents the temperature. 
For the spatial discretization, we use again second order central differences:

$$\frac{\partial}{\partial t}u(x_i,t) = \alpha \frac{u(x_{i+1},t) - 2u(x_i,t) + u(x_{i+1},t)}{\Delta x^2}.$$

The problem is how to discretize the time derivative. One guess is to use the simple explicit Euler method, which leads to:

$$ \frac{u_i^{n+1} - u_i^n}{\Delta t} = \alpha \frac{u_{i+1}^{n} - 2u_i^{n} + u_{i+1}^{n}}{\Delta x^2}.$$

where, as is standard, the subscript denotes the spatial and the superscript the time grid point. This scheme, known as [**Forward-Time Centered-Space**](https://en.wikipedia.org/wiki/FTCS_scheme),<br> 
turns out to be only conditionally stable, which means that the time step has to be smaller than a certain value, which depends on $\alpha$ and the spatial step size, to make the solution stable. We will instead use the implicit, or backward, Euler method. The resulting equation is:

$$ \frac{u_i^{n+1} - u_i^n}{\Delta t} = \alpha \frac{u_{i+1}^{n+1} - 2u_i^{n+1} + u_{i+1}^{n+1}}{\Delta x^2},$$

Since we use backward Euler in time and central differences in space, this is often called the **Backward-Time Centered-Space (BTCS) scheme**. It is unconditionally stable, but only first order accurate in time (and second order in space). A popular alternative is the [Crank Nicolson method](https://en.wikipedia.org/wiki/Crank%E2%80%93Nicolson_method), which is second order in both time and space.

We rearrange this equation to express the unknown values in terms of the known quantities:

$$ u_i^{n+1}(1+2C) - C(u_{i+1}^{n+1} + u_{i-1}^{n+1}) = u_i^n,$$

where we have defined the Courant (or CFL) number $C \equiv \alpha\Delta t/\Delta x^2$. It is important to note that the above method is numerically stable for any value of $|C|$, while the FTCS method would be stable only for $|C|\leq 0.5$, placing restrictive conditions on the choice of a time step and grid size. 

To time step $u(x,t)$ we need to solve again a linear system of equations (here, the matrix is tridiagonal).
In addition to the boundary conditions for $u$, we also need an initial condition for $u(x,0)$ at time $t=t_0$ to obtain a well-posed problem. We solve the 1D heat equation using the initial condition

$$u(t=0) = 5\cos(\pi x)$$

and the boundary conditions $u(x=0) = 2$ and $u(x=1) = 10$. We choose $\alpha = 0.005$, and solve the heat equation from $t=0$ to $t=100$ (the units are not important here, but you can think that everything is in SI units).

In [ ]:
alpha = 0.005
Nx = 100
dx = 1/Nx
x = np.linspace(0, 1, Nx + 1)

Nt = 10000
dt = 100/Nt # change to 1000, C=5 , method still works.

# Boundary condition
ua = 2
ub = 10

C = alpha*dt/dx**2
print("Courant number C =", C)

### Construct solution matrix
A = np.zeros([Nx + 1, Nx + 1])

# Diagonals equal 1+2C
for i in range(Nx + 1):
    A[i, i] = 1 + 2*C

# sub / super-diagonals equal -C
    if i > 0:
        A[i, i - 1] = -C
    if i < Nx:
        A[i, i + 1] = -C

# Construct boundary condition matrix b where only first and last term are non-zero:
b = np.zeros(Nx + 1)
b[0] = C*ua
b[-1] = C*ub

We now iterate over time, and solve the system of equations at each time-step. We plot the solution at regular time-intervals.

In [ ]:
# Intitial condition
u = np.zeros([Nt+1, Nx+1])
u[0,:] = 5*np.cos(np.pi*x)

plt.figure()
plt.plot(x, u[0],label='t = 0')

for n in range(Nt):
    
    # rewrite to take advantage of tridiagonal matrix
    u[n+1] = linalg.solve(A, b+u[n])
    
    # plot every 2000 time steps
    if ((n%2000 == 0 and n != 0) or n==500):
        
        time = n*dt
        plt.plot(x, u[n+1,:],label='t = '+str(time))
        plt.legend()
      
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,ncol=3, borderaxespad=0.)    
plt.xlabel('x')
plt.ylabel('Temperature')
plt.xlim([0, 1])
plt.show()

We see that the solution evolves towards the expected trend - a straight line starting at 2 at $x=0$ and ending at 10 at $x=1$. The heat equation tends to remove any curvature from the temperature profile, since the time derivative is proportional to the curvature. If you want to know more about the heat equation, here is an excellent introductory [YouTube video on the diffusion equation](https://www.youtube.com/watch?v=ly4S0oi3Yz8).

## Summary

* Learned about the different types of PDEs.
* Used finite-differences to solve the Laplace equation in 2D (elliptic) 
* Solved the Heat equation in 1D (parabolic) using the BTCS method.<br> Discussed FTCS as a conditionally stable alternative and Crank-Nicolson as a more accurate method.

## References

There are excellent resources for studying PDEs beyond the scope of this course:

1. [Github repository on "Practical Numerical Methods with Python" by Lorena Barba and Ian Hawke,](https://github.com/numerical-mooc/numerical-mooc) with Jupyter notebooks and a pdf with lectures on PDEs. The notebooks and lectures are recommended as a "practical" introduction to PDEs.
2. [Collection of python codes to solve PDEs](http://people.bu.edu/andasari/courses/numericalpython/python.html) from Boston University.
3. Books on PDEs:<br>
-- H. P. Langtangen, Finite Difference Computing with PDEs: A Modern Software Approach (Springer).<br>
-- W. Ames, Numerical Methods for Partial Differential Equations, 3rd Ed. (Academic Press).<br>
-- S. Mazumder, Numerical Methods for Partial Differential Equations: Finite Difference and Finite Volume Methods (Academic Press)<br>